In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import helper
import time
import datetime
import torch.nn as nn
import torchvision.models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
import torchvision.utils
import torch
import pandas as pd
from torchinfo import summary
from PIL import Image
from torchvision.transforms import ToTensor
from glob import glob
from torch.utils.data import Dataset, DataLoader, random_split
from copy import copy
from collections import defaultdict
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
from sklearn.metrics import classification_report
from tqdm import tqdm
import math
from torcheval.metrics import BinaryAccuracy
import os
import timm
import cv2
import segmentation_models_pytorch as smp
import random
from sklearn.model_selection import train_test_split
from transformers import SegformerImageProcessor, AutoModelForSemanticSegmentation
from scipy.ndimage import gaussian_filter
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")
batch_size=1
img_size=1024
class_list=['NT_epithelial',
 'NT_Muscle',
 'NT_stroma',
 'NT_immune',
 'NT_intestinal_metaplasia']
tf = ToTensor()
topilimage = torchvision.transforms.ToPILImage()
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

In [ ]:
img_path='../../data/area_segmentation/STNT/image/'
img_list=glob(img_path+'*.jpeg')
mask_list=[i.replace('/image/','/mask/npy/') for i in img_list]
mask_list=[i.replace('.jpeg','.npy') for i in mask_list]
train_img_list,test_img_list,train_mask_list,test_mask_list=train_test_split(img_list,mask_list,test_size=0.2,random_state=42)

test_image=torch.zeros((len(test_img_list),3,img_size,img_size))
test_mask=torch.zeros((len(test_img_list),len(class_list)+1,img_size,img_size),dtype=torch.float32)    

for i in tqdm(range(len(test_img_list))):
    test_image[i] = tf(Image.open(test_img_list[i]))
    np_mask=np.load(test_mask_list[i]).astype(np.uint8)
    one_hot_mask = np.zeros((1024, 1024, len(class_list)+1), dtype=np.int32)

        # 각 위치에 대해 mask 값에 해당하는 채널에 1을 설정
    np.put_along_axis(one_hot_mask, np_mask[..., np.newaxis], 1, axis=2)
    np_mask=one_hot_mask
    test_mask[i]=tf(np_mask)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_list, label_list):
        self.img_path = image_list
        self.label = label_list
        
    def trans(self,image,label):
        if random.random() > 0.5:
            transform = transforms.RandomHorizontalFlip(1)
            label = transform(label)
            image = transform(image)
            
        if random.random() > 0.5:
            transform = transforms.RandomVerticalFlip(1)
            label = transform(label)
            image = transform(image)
            
        return image,label
    
    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        image_path,label_path = self.img_path[idx],self.label[idx]

        return image_path, label_path

test_dataset = CustomDataset(test_image, test_mask)

test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [ ]:
model = smp.DeepLabV3Plus(
        encoder_name="efficientnet-b7",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=len(class_list)+1,                      # model output channels (number of classes in your dataset) 
    ).to(device)
summary(model,(batch_size,3,img_size,img_size))
def dice_loss(pred, target, num_classes=len(class_list)+1):
    smooth = 1e-6
    dice_per_class = torch.zeros((len(pred),num_classes)).to(pred.device)
    pred=F.softmax(pred,dim=1)
    for i in range(len(pred)):
        for class_id in range(num_classes):
            pred_class = pred[i, class_id, ...]
            target_class = target[i, class_id, ...]
            
            intersection = torch.sum(pred_class * target_class)
            A_sum = torch.sum(pred_class * pred_class)
            B_sum = torch.sum(target_class * target_class)
            dice_per_class[i,class_id] =(2. * intersection + smooth) / (A_sum + B_sum + smooth)

    return 1-dice_per_class

model.load_state_dict(torch.load('../../model/areaSeg/STNT_callback.pt'))

In [ ]:
def smooth_multiclass_mask(mask, sigma=5):
    """
    다중 클래스 세그멘테이션 마스크를 부드럽게 만드는 함수입니다.

    Parameters:
        mask (np.ndarray): softmax를 적용한 다중 클래스 세그멘테이션 마스크.
                           shape은 (H, W, num_classes)이어야 합니다.
        sigma (float): Gaussian 블러의 표준 편차. 값이 클수록 마스크가 더 부드럽게 됩니다.

    Returns:
        np.ndarray: 부드럽게 처리된 softmax 마스크.
    """
    # 각 클래스 채널에 대해 Gaussian 블러 적용
    smooth_mask = np.zeros_like(mask)
    for i in range(mask.shape[-1]):
        smooth_mask[:, :, i] = gaussian_filter(mask[:, :, i], sigma=sigma)

    # 각 픽셀에 대해 softmax 재적용
    smooth_mask = np.exp(smooth_mask) / np.sum(np.exp(smooth_mask), axis=-1, keepdims=True)

    return smooth_mask

topilimage = torchvision.transforms.ToPILImage()
k = cv2.getStructuringElement(cv2.MORPH_CROSS, (7,7))
train_loss_list=[]
val_loss_list=[]
train_acc_list=[]
val_acc_list=[]
MIN_loss=5000
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-4)
metrics = defaultdict(float)
model.eval()
count=0
val_running_loss=0.0
acc_loss=0
with torch.no_grad():
    for x, y in test_dataloader:
        y = y.to(device).float()
        count+=1
        x=x.to(device).float()
        predict = model(x).to(device)
        cost = dice_loss(predict, y).cpu().numpy()
        y = y.to('cpu')
        x=x.to('cpu')
        val_loss_list.append(1-cost)
        if count%5==0:
            pred_softmax = np.array(F.softmax(predict, dim=1).cpu())
            mask=np.zeros((img_size,img_size,3))
            mask[...,0]=cv2.morphologyEx(np.where(pred_softmax.argmax(axis=1)==1,255,0).astype(np.uint8)[0], cv2.MORPH_OPEN, k)
            mask[...,1]=cv2.morphologyEx(np.where(pred_softmax.argmax(axis=1)==2,255,0).astype(np.uint8)[0], cv2.MORPH_OPEN, k)
            mask[...,2]=cv2.morphologyEx(np.where(pred_softmax.argmax(axis=1)==3,255,0).astype(np.uint8)[0], cv2.MORPH_OPEN, k)
            mask[...,1]+=cv2.morphologyEx(np.where(pred_softmax.argmax(axis=1)==4,255,0)[0].astype(np.uint8), cv2.MORPH_OPEN, k)
            mask[...,2]+=cv2.morphologyEx(np.where(pred_softmax.argmax(axis=1)==4,255,0)[0].astype(np.uint8), cv2.MORPH_OPEN, k)
            mask[...,0]+=cv2.morphologyEx(np.where(pred_softmax.argmax(axis=1)==5,255,0)[0].astype(np.uint8), cv2.MORPH_OPEN, k)
            mask[...,2]+=cv2.morphologyEx(np.where(pred_softmax.argmax(axis=1)==5,255,0)[0].astype(np.uint8), cv2.MORPH_OPEN, k)
            label=np.zeros((img_size,img_size,3))
            label[...,0]=np.where(y.argmax(dim=1)==1,255,0)
            label[...,1]=np.where(y.argmax(dim=1)==2,255,0)
            label[...,2]=np.where(y.argmax(dim=1)==3,255,0)
            label[...,1]+=np.where(y.argmax(dim=1)==4,255,0)[0]
            label[...,2]+=np.where(y.argmax(dim=1)==4,255,0)[0]
            label[...,0]+=np.where(y.argmax(dim=1)==5,255,0)[0]
            label[...,2]+=np.where(y.argmax(dim=1)==5,255,0)[0]
            img=topilimage(x[0])
            overlay=np.array(img)*0.7+np.array(mask)*0.3
            overlay=Image.fromarray(overlay.astype('uint8'))
            fig = plt.figure(figsize=(20,30))
            rows = 1
            cols = 4
            ax1 = fig.add_subplot(rows, cols, 1)
            ax1.imshow(img)
            ax1.set_title('image')
            ax1.axis("off")
            
            ax2 = fig.add_subplot(rows, cols, 2)
            ax2.imshow(label)
            ax2.set_title('label')
            ax2.axis("off")
            
            ax3 = fig.add_subplot(rows, cols, 3)
            ax3.imshow(mask)
            ax3.set_title('predict')
            ax3.axis("off")
            
            ax4 = fig.add_subplot(rows, cols, 4)
            ax4.imshow(overlay)
            ax4.set_title('overlay')
            ax4.axis("off")
            plt.show()
           

print('batch size= 4')
print('image size= 224')
print('learning rate= 0.0001')

In [ ]:
cv2.morphologyEx(np.where(pred_softmax.argmax(axis=1)==1,255,0).astype(np.uint8)[0], cv2.MORPH_OPEN, k)

In [ ]:
dice_score=np.array(val_loss_list)
print(f'{class_list[0]} Class DSC={dice_score[...,1].mean()}')
print(f'{class_list[1]} Class DSC={dice_score[...,2].mean()}')
print(f'{class_list[2]} Class DSC={dice_score[...,3].mean()}')
print(f'{class_list[3]} Class DSC={dice_score[...,4].mean()}')
print(f'{class_list[4]} Class DSC={dice_score[...,5].mean()}')





: 

In [ ]:
from shapely.geometry import Polygon, MultiPolygon
import cv2
def binary_mask_to_polygon(binary_mask):
    # binary_mask는 2차원 numpy array여야 합니다.
    # Contours를 찾습니다.
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    polygons = []
    for contour in contours:
        # 각 contour를 polygon으로 변환
        if len(contour) >= 3:  # 유효한 polygon을 만들기 위해서 최소한 3개의 점이 필요합니다.
            poly = Polygon(shell=[(point[0][0], point[0][1]) for point in contour])
            polygons.append(poly)
    
    if len(polygons) > 1:
        # 여러 개의 polygon이 있을 경우 MultiPolygon으로 변환
        return polygons
    elif len(polygons) == 1:
        return polygons[0]
    else:
        return None
    
poly=binary_mask_to_polygon(np.array(mask)[...,1])
polygon_arrays = []
for polygon in poly.geoms:
    exterior_coords = np.array(polygon.exterior.coords)
    polygon_arrays.append(exterior_coords)

# 변환된 numpy 배열 출력
for i, array in enumerate(polygon_arrays):
    print(f"Polygon {i+1} coordinates:")
    print(array)

In [ ]:
for polygon in poly:
    print(polygon)

In [ ]:
from skimage.draw import polygon2mask
mask1=np.zeros((1024,1024))
for i in range(len(polygon_arrays)):S
    coordinate=np.array(polygon_arrays[i])
    coordinate[:,[0, 1]]=coordinate[:,[1, 0]]
    mask_temp=polygon2mask((1024,1024), coordinate)
    mask1+=mask_temp
ax=plt.figure(figsize=(10,10))
ax.add_subplot(1,3,1)
plt.imshow(img)
ax.add_subplot(1,3,2)
plt.imshow(mask1)
ax.add_subplot(1,3,3)
img1=np.array(img)
img1[...,0]=img1[...,0]*0.8+mask1*255*0.2
plt.imshow(img1)

In [ ]:
len(polygon_arrays)

In [ ]:
poly